# Debugging pomagma.reducer.bohm

There appears to a bug in `reducer.bohm`, exercised with `lib.num_test` and other uses of recursion.

In [1]:
from pomagma.reducer import syntax, sugar, lib, bohm, church, curry
from pomagma.reducer.engines import engine

def convert(code):
    code = sugar.as_code(code)
    return bohm.convert(code)

def app(*args):
    args = map(convert, args)
    result = args[0]
    for arg in args[1:]:
        result = bohm.app(result, arg)
    return result

def pretty(code):
    code = sugar.as_code(code)
    code = church.convert(code)
    return syntax.sexpr_print(code)

def print_(code):
    print(pretty(code))

## Example: `lib.num_test(two)`

In [2]:
zero = convert(lib.zero)
print_(zero)

(FUN a (FUN b a))


In [3]:
succ = convert(lib.succ)
print_(succ)

(FUN a (FUN b (FUN c (c a))))


In [4]:
one = convert(app(succ, zero))
print_(one)

(FUN a (FUN b (b (FUN c (FUN d c)))))


In [5]:
two = convert(app(succ, one))
print_(two)

(FUN a (FUN b (b (FUN c (FUN d (d (FUN e (FUN f e))))))))


In [6]:
f,g,x,y = map(syntax.NVAR, 'fgxy')
print_(app(succ, x, f, g))

(g x)


In [7]:
num_test = convert(lib.num_test)
print_(num_test)

(FUN a (UNIT (a (FUN b b) (FUN c (a a) (FUN d (FUN e (UNIT (e (FUN f f) (d d)))))))))


## Example: `lib.fix` and other fixed-point combinators.

In [8]:
print_(convert(lib.fix))

(FUN a (a (FUN b (a a) (FUN c (FUN d (d (c c d)))) a)))


Let's create a simple fixed point combinator.

In [9]:
Y = sugar.as_code(lambda f: sugar.app(lambda x: sugar.app(x, x),
                                      lambda x: sugar.app(f, sugar.app(x, x))))
print_(Y)

(B (S I I) (C B (S I I)))


In [10]:
for i in range(17):
    print_(curry.reduce(syntax.APP(Y, f), i))

(B (S I I) (C B (S I I)) f)
(S I I (C B (S I I) f))
(I (C B (S I I) f) (I (C B (S I I) f)))
(C B (S I I) f (I (C B (S I I) f)))
(B f (S I I) (I (C B (S I I) f)))
(f (S I I (I (C B (S I I) f))))
(f (I (I (C B (S I I) f)) (I (I (C B (S I I) f)))))
(f (I (C B (S I I) f) (I (I (C B (S I I) f)))))
(f (C B (S I I) f (I (I (C B (S I I) f)))))
(f (B f (S I I) (I (I (C B (S I I) f)))))
(f (f (S I I (I (I (C B (S I I) f))))))
(f (f (I (I (I (C B (S I I) f))) (I (I (I (C B (S I I) f)))))))
(f (f (I (I (C B (S I I) f)) (I (I (I (C B (S I I) f)))))))
(f (f (I (C B (S I I) f) (I (I (I (C B (S I I) f)))))))
(f (f (C B (S I I) f (I (I (I (C B (S I I) f)))))))
(f (f (B f (S I I) (I (I (I (C B (S I I) f)))))))
(f (f (f (S I I (I (I (I (C B (S I I) f))))))))


...which seems to beta-reduce as expected.

Next let's convert from combinatory algebra to lambda-calculus.

In [11]:
Y2 = convert(Y)
print(syntax.sexpr_print(Y2))
print_(Y2)
print_(bohm.app(Y2, f))

(ABS (ABS (1 0 (1 0)) (ABS (ABS (1 (0 0))))))
(FUN a (FUN b (a b (a b)) (FUN c (FUN d (c (d d))))))
(FUN a (f a (f a)) (FUN b (FUN c (b (c c)))))


In [12]:
for i in range(4):
    print_(bohm.reduce(bohm.app(Y2, f), i))

(FUN a (f a (f a)) (FUN b (FUN c (b (c c)))))
(f (FUN a (FUN b (a (b b)))) (f (FUN c (FUN d (c (d d))))))
(f (FUN a (FUN b (a (b b)))) (f (FUN c (FUN d (c (d d))))))
(f (FUN a (FUN b (a (b b)))) (f (FUN c (FUN d (c (d d))))))


...which does not beta-reduce as expected.

So far the bug appears to be in `bohm.convert(-)`. Let's try converting back to combinatory algebra.

In [13]:
Y3 = curry.convert(Y2)
print_(Y3)

(C (S S I) (C B (S I I)))


In [14]:
for i in range(7):
    print_(curry.reduce(syntax.APP(Y3, f), i))

(C (S S I) (C B (S I I)) f)
(S S I f (C B (S I I)))
(S f (I f) (C B (S I I)))
(f (C B (S I I)) (I f (C B (S I I))))
(f (C B (S I I)) (f (C B (S I I))))
(f (C B (S I I)) (f (C B (S I I))))
(f (C B (S I I)) (f (C B (S I I))))


...which again fails to beta-reduce as expected.

Let's try `bohm.convert`ing simpler terms.

In [15]:
def show(sexpr, steps=0, message=''):
    code = syntax.sexpr_parse(sexpr)
    code = bohm.reduce(code, steps)
    sexpr = syntax.sexpr_print(code)
    print('{}  # {}'.format(sexpr, message) if message else sexpr)

show('(S I I)')
show('(S I I x)')
show('(B (S I I))')
show('(B (S I I) fxx)')
show('(B (S I I) (C B (S I I)))', message='FIXME there is an extra ABS on the right')
show('(B (S I I) (C B (S I I)) f)')
show('(S I I (C B (S I I) f))', message='This is correct.')
show('(B (S I I) (C B (S I I)) f)', 1, message='FIXME B reduction fails')
show('(B (S I I) (C B (S I I)) f)', 2)
show('(B (S I I) (C B (S I I)) f)', 3)
show('(B (S I I) (C B (S I I)) f)', 4)

(ABS (0 0))
(x x)
(ABS (ABS (1 0 (1 0))))
(ABS (fxx 0 (fxx 0)))
(ABS (ABS (1 0 (1 0)) (ABS (ABS (1 (0 0))))))  # FIXME there is an extra ABS on the right
(ABS (f 0 (f 0)) (ABS (ABS (1 (0 0)))))
(ABS (0 0) (ABS (f (0 0))))  # This is correct.
(f (ABS (ABS (1 (0 0)))) (f (ABS (ABS (1 (0 0))))))  # FIXME B reduction fails
(f (ABS (ABS (1 (0 0)))) (f (ABS (ABS (1 (0 0))))))
(f (ABS (ABS (1 (0 0)))) (f (ABS (ABS (1 (0 0))))))
(f (ABS (ABS (1 (0 0)))) (f (ABS (ABS (1 (0 0))))))


Now if we avoid abstracting out `f`, bohm.convert works fine:

In [16]:
Yf = sugar.as_code(sugar.app(lambda x: sugar.app(x, x),
                             lambda x: sugar.app(f, sugar.app(x, x))))
print_(Yf)
for i in range(5):
    print_(bohm.reduce(Yf, i))

(S I I (B f (S I I)))
(FUN a (a a) (FUN b (f (b b))))
(f (FUN a (a a) (FUN b (f (b b)))))
(f (f (FUN a (a a) (FUN b (f (b b))))))
(f (f (f (FUN a (a a) (FUN b (f (b b)))))))
(f (f (f (f (FUN a (a a) (FUN b (f (b b))))))))


In [17]:
Y_f = syntax.sexpr_parse('(B (S I I) (C B (S I I)) f)')
for i in range(6):
    print_(curry.reduce(Y_f, i))

(B (S I I) (C B (S I I)) f)
(S I I (C B (S I I) f))
(I (C B (S I I) f) (I (C B (S I I) f)))
(C B (S I I) f (I (C B (S I I) f)))
(B f (S I I) (I (C B (S I I) f)))
(f (S I I (I (C B (S I I) f))))
